In [ ]:
#Libraries to install: 

# !pip install snowflake-connector-python
# !pip install tableauhyperapi
# !pip install tableau-api-lib
# !pip install smartsheet-python-sdk
# !pip install bcolors

# 1. Check pending accounts:

In [ ]:
import os, datetime, glob, pandas as pd, numpy as np, shutil, itertools, snowflake.connector, re, importlib, utils_lib as utils, requests, json, webbrowser#, smartsheet_lib as smartsheet
class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

auth_path = r".\credentials.json"
auth_path = auth_path.replace("\\","/")

with open(auth_path) as json_file:
    json_credentials = json.load(json_file)
    json_file.close()

user = json_credentials["cisco_tableau"]["user"] # Your Cisco e-mail address
cs,cnn = utils.connec_to_sf(user)
tableau_template_name = json_credentials["paths"]["tableau_template_oe_path"] #twb template name/path
ib_file_name = 'IB.csv'
coverage_file_name = 'Coverage.csv'
sw_file_name = 'SW.csv'
sntc_mapping_path = json_credentials["paths"]["sntc_mapping_path"]
personal_access_token_name = json_credentials["cisco_tableau"]["access_token_name"]
personal_access_token_secret = json_credentials["cisco_tableau"]["token"]

check_tableau_log_in=webbrowser.open("cisco tableau url")

fields_df = utils.get_OE_pending_requests(user,cs)
fields_df[['OE_KEY', 'REQUEST_ID', 'DATE_CREATED', 'ASSIGNED_DA', 'CAMPAIGN_NAME','CUSTOMER_NAME','ID_TYPE', 'SAV_ID', 'CAV_ID','CAV_BU_ID', 'GU_ID', 'LVL1', 'LVL2', 'CONTRACT_ID', 'INVENTORY_NAME','APPLIANCE_ID', 'CR_PARTY_NAME', 'CR_PARTY_ID','DA_COMMENTS', 'STATUS', 'REQUESTER_NAME', 'OP_STATUS', 'DA_ASSIGNED_DATE', 'OP_COMPLETE_DATE','OA_PACKAGE_TYPE', 'OA_URL', 'REJECTED_REASON']]

# 2. Data from Snowflake:

In [ ]:
# IDs list for Snowflake (separator must be ',')
savs_list,gu_list,cav_list,cr_list = utils.get_ids_list(fields_df,separator=',')

# 1) Downloading ib data from snowflake
ib_df = utils.get_ib_data(user,savs_list,gu_list,cr_list,cav_list,cs,1)
ib_df.loc[:,'Default Service List Price USD':'Asset List Amount'] = ib_df.loc[:,'Default Service List Price USD':'Asset List Amount'].fillna(0)

# 2) Downloading Coverage data from snowflake
coverage_df = utils.get_coverage_data(user,savs_list,gu_list,cr_list,cav_list,cs,2)
coverage_df.loc[:,'Item Quantity':'Annualized Extended Contract Line List USD Amount'] = coverage_df.loc[:,'Item Quantity':'Annualized Extended Contract Line List USD Amount'].fillna(0)

# 3) Downloading TAC data from snowflake
tac_df = utils.get_tac_df_new(user,savs_list,gu_list,cr_list,cav_list,cs, 3)
tac_df["BUG_CNT"] = tac_df["BUG_CNT"].replace("",0.0)
tac_df["HYBRID_PRODUCT_FAMILY"] = tac_df['RU_BK_PRODUCT_FAMILY_ID'] # replace column data
tac_df.drop(columns='RU_BK_PRODUCT_FAMILY_ID',axis=1,inplace=True)

# 4) Downloading DNA data from snowflake
dna_df = utils.get_dna_df(user,savs_list,gu_list,cr_list,cav_list,cs, 4)

# 5) 6) Downloading EA data from snowflake
EA_details_df = utils.get_EA_details_data_(user,savs_list,gu_list,cr_list,cav_list,cs,5)
EA_eligibility_df = utils.get_EA_eligibility_data_(user,savs_list,gu_list,cr_list,cav_list,cs,6)

# 7) Telemery dataframe from Snowflake (may take a few minutes)
telemetry_df = utils.get_telemetry_df2(user,savs_list,gu_list,cr_list,cav_list,cs,7)

# 3. Data transformation:

In [ ]:
# ------------------------------------------------------------------------------------------------ data cleaning
renew_filtered, coverage_filtered, tac_filtered,telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered = ib_df.copy(), coverage_df.copy(), tac_df.copy(), telemetry_df.copy(), dna_df.copy(), EA_details_df.copy(), EA_eligibility_df.copy()
renew_filtered, coverage_filtered, tac_filtered,telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered = utils.oe_data_cleaning(renew_filtered, coverage_filtered, tac_filtered,telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered)

# --------------------------------------------------- Creating dataframe with all REQUEST_ID and accounts id

savs, gus, cavs, crs, request = utils.split_account_ids(fields_df)

# --------------------------------------------------- Adding Request ID and Secret ID columns

renew_filtered, coverage_filtered, tac_filtered, telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered = utils.add_OE_KEY(renew_filtered, coverage_filtered, tac_filtered, telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered, request)

# --------------------------------------------------- Adapting data

renew_filtered, coverage_filtered, tac_filtered, telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered = utils.adapt_OE_data(renew_filtered, coverage_filtered, tac_filtered, telemetry_filtered, dna_filtered, EA_details_filtered, EA_eligibility_filtered)

In [ ]:
#Optional
importlib.reload(utils)
#pd.options.display.max_columns=99 #To see more columns in the dataframes

# QA

In [ ]:
validation_data = utils.ib_values_validation(utils.IB_attributes(renew_filtered,coverage_filtered))
validation_data = validation_data.merge(utils.oppty_validation(utils.smartnet_verification(renew_filtered)), how='left', left_on='OE_KEY', right_on='OE_KEY')
validation_data = validation_data.merge(utils.lenght_validation(utils.smartnet_total_care_NBD_list_price(renew_filtered)).rename(columns={'Lenght':'Smartnet value lenght'}), how='left', left_on='OE_KEY', right_on='OE_KEY')
validation_data = validation_data.merge(utils.lenght_validation(utils.estimated_list_price(renew_filtered)).rename(columns={'Lenght':'Success_L2 value lenght'}), how='left', left_on='OE_KEY', right_on='OE_KEY')
validation_data = validation_data.merge(fields_df[['OE_KEY','CUSTOMER_NAME']].astype({'OE_KEY':str}), how='left', left_on='OE_KEY', right_on='OE_KEY')
validation_data.loc[:,'OE_KEY':'CUSTOMER_NAME'].style.applymap(lambda x: utils.color_qa(x))

# 4. Uploading data into snowflake

In [ ]:
# ------------------------------------------------------------ Uploading dataframes into snowflake

# ----------------------------- IB 

utils.upload_data_to_sf(renew_filtered, user, 'RT_TWB_OE_IB', 'IB',cnn)

# ----------------------------- Coverage

utils.upload_data_to_sf(coverage_filtered, user, 'RT_TWB_OE_COVERAGE', 'COVERAGE',cnn)

# ----------------------------- TAC

utils.upload_data_to_sf(tac_filtered, user, 'RT_TWB_OE_TAC', 'TAC',cnn)

# ----------------------------- Telemetry

utils.upload_data_to_sf(telemetry_filtered, user, 'RT_TWB_OE_TELEMETRY', 'TELEMETRY',cnn)

# ----------------------------- DNA

utils.upload_data_to_sf(dna_filtered, user, 'RT_TWB_OE_DNA', 'DNA',cnn)

# ----------------------------- EA

utils.upload_data_to_sf(EA_details_filtered, user, 'RT_TWB_EA_DETAILS', 'EA Details',cnn)
utils.upload_data_to_sf(EA_eligibility_filtered, user, 'RT_TWB_EA_ELIGIBILITY', 'EA Eligibility',cnn)

# 5. Updating Tracker

In [ ]:
# ------------------------------------------------------------ OP Intake Tracker
op_tracker=utils.update_OE_tracker(user, fields_df, renew_filtered, telemetry_filtered)
op_tracker[['OE_KEY', 'REQUEST_ID', 'DATE_CREATED', 'ASSIGNED_DA', 'CAMPAIGN_NAME','CUSTOMER_NAME','ID_TYPE', 'SAV_ID', 'CAV_ID','CAV_BU_ID', 'GU_ID', 'LVL1', 'LVL2', 'CONTRACT_ID', 'INVENTORY_NAME','APPLIANCE_ID', 'CR_PARTY_NAME', 'CR_PARTY_ID','DA_COMMENTS', 'STATUS', 'REQUESTER_NAME', 'OP_STATUS', 'DA_ASSIGNED_DATE', 'OP_COMPLETE_DATE','OA_PACKAGE_TYPE', 'OA_URL', 'REJECTED_REASON','UPLOADED_TIME']]

# 6. Upload to Compass Request Tracker:

In [ ]:
op_tracker=utils.upload_to_compass_tracker(op_tracker)

# 7. Uploading Tracker to Snowflake

In [ ]:
utils.print_smartsheet_data(op_tracker)

In [ ]:
utils.upload_data_to_sf(op_tracker, user, 'RT_OP_INTAKE_REQUEST_DETAILS', 'Tracker',cnn)

In [ ]:
#Optional
importlib.reload(utils)
#pd.options.display.max_columns=99 #To see more columns in the dataframes